## Experimental Study of the Marshall–Olkin Joint Survival Function

*(Lecture series by Professor Fabio Gobbi presented in the Credit Risk Modeling course, University of Siena; see Gobbi et al. 2019, 2021 for related publications.)*

The following Python script reproduces an **experiment** illustrating how changes in the systemic hazard λ₁₂ affect both joint survival and joint default probabilities. All steps and formulas remain fully parameterized in terms of λ₁, λ₂, and λ₁₂.


In [6]:
import numpy as np

# 1. Define the shock intensity parameters
lambda_1  = 0.5     # idiosyncratic hazard for obligor X
lambda_2  = 1.0     # idiosyncratic hazard for obligor Y
lambda_12 = 0.25    # systemic hazard affecting both X and Y

# 2. Marginal survival functions
#    X ∼ Exponential(λ₁ + λ₁₂), Y ∼ Exponential(λ₂ + λ₁₂)
def Sx(x):
    """
    Marginal survival function of X:
      Sx(x) = P(X > x) = exp[-(λ₁ + λ₁₂) x]
    """
    return np.exp(-(lambda_1 + lambda_12) * x)

def Sy(y):
    """
    Marginal survival function of Y:
      Sy(y) = P(Y > y) = exp[-(λ₂ + λ₁₂) y]
    """
    return np.exp(-(lambda_2 + lambda_12) * y)

# 3. Copula parameters α and β
#    α = λ₁₂ / (λ₁ + λ₁₂),  β = λ₁₂ / (λ₂ + λ₁₂)
def compute_alpha_beta(l1, l2, l12):
    """
    Compute the weights of the systemic shock:
      α = systemic / total hazard of X
      β = systemic / total hazard of Y
    """
    alpha = l12 / (l1 + l12)
    beta  = l12 / (l2 + l12)
    return alpha, beta

alpha, beta = compute_alpha_beta(lambda_1, lambda_2, lambda_12)
print(f"Computed α = {alpha:.4f}, β = {beta:.4f}")

# 4. General joint survival function H(x, y)
#    Derived from:
#      H(x,y) = exp[-λ₁ x - λ₂ y - λ₁₂ max(x,y)]
#    Equivalently:
#      H(x,y) = exp[-(λ₁+λ₁₂) x - (λ₂+λ₁₂) y + λ₁₂⋅min(x,y)]
def H(x, y):
    """
    Joint survival P(X > x, Y > y):
      = exp[-(λ₁+λ₁₂)x - (λ₂+λ₁₂)y + λ₁₂ * min(x, y)]
    """
    # total hazards for marginals
    total_hazard_x = lambda_1 + lambda_12
    total_hazard_y = lambda_2 + lambda_12

    # compute the base exponent terms
    term_x = - total_hazard_x * x
    term_y = - total_hazard_y * y

    # add back the systemic shock contribution on the overlap
    overlap = lambda_12 * min(x, y)

    return np.exp(term_x + term_y + overlap)

# 5. Example evaluation at t = 3
t = 3.0
joint_survival = H(t, t)
print(f"H(3,3) = {joint_survival*100:.4f}%  # P(X>3, Y>3)")

# 6. Marginal default probabilities F_X(3), F_Y(3)
Fx_t = 1 - Sx(t)
Fy_t = 1 - Sy(t)
print(f"F_X(3) = {Fx_t:.6f}, F_Y(3) = {Fy_t:.6f}")

# 7. Joint default probability via inclusion–exclusion
#    P(X ≤ t, Y ≤ t) = 1 - Sx(t) - Sy(t) + H(t,t)
joint_default = 1 - Sx(t) - Sy(t) + joint_survival
print(f"Joint default P(X≤3, Y≤3) = {joint_default*100:.2f}%")


Computed α = 0.3333, β = 0.2000
H(3,3) = 0.5248%  # P(X>3, Y>3)
F_X(3) = 0.894601, F_Y(3) = 0.976482
Joint default P(X≤3, Y≤3) = 87.63%


## Steps

1. **Parameters**  
   – λ₁, λ₂ are the idiosyncratic hazard rates for obligors X and Y.  
   – λ₁₂ is the systemic hazard rate that can simultaneously terminate both lifetimes.

2. **Marginals**  
   Each obligor’s lifetime is the minimum of its idiosyncratic shock and the systemic shock.  Thus  
   $$
     S_X(x) = e^{-(\lambda_1 + \lambda_{12})\,x},\quad
     S_Y(y) = e^{-(\lambda_2 + \lambda_{12})\,y}.
   $$

3. **Copula Weights**  
   $$\alpha$$ and $$\beta$$ measure the proportion of each obligor’s total hazard attributable to the systemic shock:
   $$
     \alpha = \frac{\lambda_{12}}{\lambda_1 + \lambda_{12}},\quad
     \beta  = \frac{\lambda_{12}}{\lambda_2 + \lambda_{12}}.
   $$

4. **Joint Survival Function**  
   The bivariate survival is
   $$
     H(x,y)
     = \exp\bigl[-\lambda_1 x - \lambda_2 y - \lambda_{12}\max(x,y)\bigr]
     = \exp\bigl[-(\lambda_1+\lambda_{12})x -(\lambda_2+\lambda_{12})y + \lambda_{12}\min(x,y)\bigr].
   $$
   The implementation uses this unified formula to avoid any hard-coded numeric exponents.

5. **Numerical Example**  
   With $$t = 3$$:
   - Joint survival $$H(3,3)\approx0.5248\%$$.  
   - Marginal defaults $$F_X(3)\approx0.894601$$, $$F_Y(3)\approx0.976482$$.  
   - Joint default $$P(X{\le}3,Y{\le}3)\approx87.63\%$$.

In this experiment, the objective is to investigate how variations in the systemic hazard λ₁₂ affect the probability that both obligors either survive beyond a fixed time or default by that time.

**References**  
- Gobbi, F., Kolev, N., & Mulinacci, S. (2019). *Joint Life Insurance Pricing Using Extended Marshall–Olkin Models*. ASTIN Bulletin, 49(2), 409–432.  
- Gobbi, F., Kolev, N., & Mulinacci, S. (2021). *Ryu-type Extended Marshall–Olkin Model with Implicit Shocks and Joint Life Insurance Applications*. Insurance: Mathematics and Economics, 98, 141–154.